In [1]:
# run ipython with this command: jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
import matplotlib
import brewer2mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout,Flatten, Activation, Dropout, Flatten
from keras.models import model_from_json
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.activations import relu, tanh, elu
from keras.backend import clear_session
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

set3 = brewer2mpl.get_map('Set3', 'qualitative', 7).mpl_colors
% matplotlib inline

Using TensorFlow backend.


In [2]:
train_data_x = pd.read_pickle('normalized_fer2013.pkl')
train_data_y = pd.read_pickle('normalized_fer2013_labels.pkl').astype(int)
test_data_x = pd.read_pickle('normalized_test_fer2013.pkl')
test_data_y = pd.read_pickle('normalized_test_fer2013_labels.pkl').astype(int)

train_data_x = train_data_x.as_matrix().reshape((-1,48,48,1))
test_data_x = test_data_x.as_matrix().reshape((-1,48,48,1))

train_data_y = train_data_y.as_matrix()
test_data_y = test_data_y.as_matrix()

train_data_y = keras.utils.to_categorical(train_data_y, num_classes=7)
test_data_y = keras.utils.to_categorical(test_data_y, num_classes=7)

Tune Batch Size and Number of Epochs

In [9]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = keras.models.Sequential()
    model.add(Conv2D(16, (5,5), strides=(1,1),activation='elu',padding='valid',input_shape=(48,48,1)))
    model.add(Conv2D(8, (5,5), strides=(1,1),activation='elu',padding='valid'))
    model.add(Conv2D(8, (5,5), strides=(1,1),activation='elu',padding='valid'))
    model.add(Conv2D(4, (5,5), strides=(1,1),activation='elu',padding='valid'))
    
    model.add(Flatten())
    model.add(Dense(64,activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(32,activation='elu'))
    model.add(Dense(16,activation='elu'))
    model.add(Dense(7,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
# split into input (X) and output (Y) variables
X = train_data_x
Y = train_data_y
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [128, 256, 1024]
epochs = [10]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose = 10)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] batch_size=128, epochs=10 .......................................
[CV]  batch_size=128, epochs=10, score=0.38181818196143225, total=33.6min
[CV] batch_size=128, epochs=10 .......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 35.0min remaining:    0.0s


[CV]  batch_size=128, epochs=10, score=0.40114942532472603, total=30.3min
[CV] batch_size=128, epochs=10 .......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 66.3min remaining:    0.0s


KeyboardInterrupt: 

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
    # create model
    model = keras.models.Sequential()
    model.add(Conv2D(16, (5,5), strides=(1,1),activation='elu',padding='valid',input_shape=(48,48,1)))
    model.add(Conv2D(8, (5,5), strides=(1,1),activation='elu',padding='valid'))
    model.add(Conv2D(8, (5,5), strides=(1,1),activation='elu',padding='valid'))
    model.add(Conv2D(4, (5,5), strides=(1,1),activation='elu',padding='valid'))
    
    model.add(Flatten())
    model.add(Dense(64,activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(32,activation='elu'))
    model.add(Dense(16,activation='elu'))
    model.add(Dense(7,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
# split into input (X) and output (Y) variables
X = train_data_x
Y = train_data_y
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=256, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose = 10)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 7 candidates, totalling 21 fits
[CV] optimizer=SGD ...................................................
[CV] ......... optimizer=SGD, score=0.34963427374729167, total=23.1min
[CV] optimizer=SGD ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 24.0min remaining:    0.0s


[CV] ......... optimizer=SGD, score=0.33416927892212583, total=20.8min
[CV] optimizer=SGD ...................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 45.5min remaining:    0.0s


[CV] ......... optimizer=SGD, score=0.35876267102896103, total=20.8min
[CV] optimizer=RMSprop ...............................................


...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 110.4min remaining:    0.0s


[CV] ..... optimizer=RMSprop, score=0.39492109929310615, total=20.8min
[CV] optimizer=Adagrad ...............................................


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 132.0min remaining:    0.0s


[CV] ...... optimizer=Adagrad, score=0.3857889237573279, total=21.1min
[CV] optimizer=Adagrad ...............................................


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed: 154.0min remaining:    0.0s
